In [ ]:
from mobgap.data import LabExampleDataset
example_data = LabExampleDataset()

In [ ]:
single_test = ha_example_data.get_subset(
    cohort="HA", participant_id="002", test="Test5", trial="Trial2"
)
single_test

In [ ]:
imu_data = single_test.data["LowerBack"]
imu_data

In [ ]:
single_sensor_data = single_test.data_ss
single_sensor_data

In [ ]:
import matplotlib.pyplot as plt

single_sensor_data.filter(like="gyr").plot()
plt.show()

In [ ]:
single_test.recording_metadata

In [ ]:
single_test.participant_metadata

In [ ]:
example_data_with_reference = LabExampleDataset(reference_system="Stereophoto")
single_trial_with_reference = example_data_with_reference.get_subset(
    cohort="HA", participant_id="002", test="Test5", trial="Trial2"
)
single_trial_with_reference.raw_reference_parameters_

In [ ]:
ref_paras = single_trial_with_reference.reference_parameters_

ref_paras.wb_list

In [ ]:
ref_paras.ic_list

In [ ]:
ref_paras.stride_parameters

In [ ]:
from mobgap.data import (
    get_all_lab_example_data_paths,
    load_mobilised_matlab_format,
)

all_example_data_paths = get_all_lab_example_data_paths()
list(all_example_data_paths.keys())

In [ ]:
example_participant_path = all_example_data_paths[("HA", "002")]
data = load_mobilised_matlab_format(example_participant_path / "data.mat")

In [ ]:
test_list = list(data.keys())
test_list

In [ ]:
test_11_data = data[test_list[2]]
imu_data = test_11_data.imu_data["LowerBack"]
imu_data

In [ ]:
data_with_reference = load_mobilised_matlab_format(
    example_participant_path / "data.mat", reference_system="INDIP"
)
selected_test = data_with_reference[test_list[2]]

In [ ]:
raw_reference_data = selected_test.raw_reference_parameters

In [ ]:
ref_sampling_rate_hz = selected_test.metadata["reference_sampling_rate_hz"]
ref_sampling_rate_hz

In [ ]:
from mobgap.data import parse_reference_parameters

data_sampling_rate_hz = selected_test.metadata["sampling_rate_hz"]

ref_paras_functional = parse_reference_parameters(
    raw_reference_data["wb"],
    data_sampling_rate_hz=data_sampling_rate_hz,
    ref_sampling_rate_hz=ref_sampling_rate_hz,
)

In [ ]:
ref_paras_functional

In [ ]:
from mobgap.data import LabExampleDataset

dataset = LabExampleDataset(reference_system="INDIP")
datapoint = dataset.get_subset(
    cohort="HA", participant_id="001", test="Test11", trial="Trial1"
)
data = datapoint.data_ss
data

In [ ]:
ref_data = datapoint.reference_parameters_
ref_data.wb_list

In [ ]:
ref_data.ic_list

In [ ]:
ref_data_rel = datapoint.reference_parameters_relative_to_wb_
ref_walking_bouts = ref_data_rel.wb_list
ref_walking_bouts

In [ ]:
ref_ics_rel = ref_data_rel.ic_list
ref_ics_rel.loc[0]  # First WB

In [ ]:
ref_ics_rel.loc[1]  

In [ ]:
from mobgap.pipeline import GsIterator

gs_iterator = GsIterator()

# For most use-cases, the default configuration of the :class:`~mobgap.pipeline.GsIterator` should be sufficient.
# This allows you to specify the following results:
gs_iterator.data_type

In [ ]:
for (wb, data_per_wb), result in gs_iterator.iterate(data, ref_walking_bouts):
    print("GS/WB id: ", wb.id)
    print(
        "Expected N-samples in wb: ",
        ref_walking_bouts.loc[wb.id].end - ref_walking_bouts.loc[wb.id].start,
    )
    print("N-samples in wb: ", len(data_per_wb))

    # We can use the wb.id to get the reference initial contacts that belong to this GS/WB
    ics_per_wb = ref_ics_rel.loc[wb.id]
    # These could be used in some algorithm.
    # Here we will just store them in the results.
    result.ic_list = ics_per_wb

In [ ]:
gs_iterator.results_.ic_list

In [ ]:
from mobgap.data import MobilisedCvsDmoDataset, get_example_cvs_dmo_data_path

example_data_base_path = get_example_cvs_dmo_data_path()
dmo_data_path = example_data_base_path / "cvs-T1-test_data.csv"
mapping_path = example_data_base_path / "cvs-T1-test_data_mapping.csv"

dataset = MobilisedCvsDmoDataset(
    dmo_path=dmo_data_path, site_pid_map_path=mapping_path
)
dataset

In [ ]:
dataset.data

In [ ]:
dataset.data_mask

In [ ]:
single_participant = dataset.get_subset(participant_id="10004")
single_participant

In [ ]:
print(single_participant.measurement_site)
print(single_participant.timezone)

In [ ]:
from pathlib import Path

from mobgap.utils.misc import get_env_var

# dataset_path = Path(get_env_var("MOBGAP_TVS_DATASET_PATH").strip('"'))


In [ ]:
from typing import Optional, Union
from mobgap.data import get_example_csv_data_path
import pandas as pd

path = get_example_csv_data_path()
all_data_files = sorted(list(path.rglob("*.csv")))
all_data_files


data = pd.read_csv(all_data_files[0])
data.head()

sampling_rate_hz = 100

In [ ]:
data

In [ ]:
from pathlib import Path

from mobgap.consts import GRAV_MS2


def load_data(path: Path) -> pd.DataFrame:
    data = pd.read_csv(path)
    data[["acc_x", "acc_y", "acc_z"]] = (
        data[["acc_x", "acc_y", "acc_z"]] * GRAV_MS2
    )
    return data


data = load_data(all_data_files[0])
data.head()

In [ ]:
from mobgap.gait_sequences import GsdIluz
from mobgap.utils.conversions import to_body_frame

gsd = GsdIluz()
gsd.detect(data=to_body_frame(data), sampling_rate_hz=sampling_rate_hz)
gsd.gs_list_

In [ ]:
import json
from pprint import pprint

def load_particpant_metadata(path: Path):
    with path.open("r") as f:
        metadata = json.load(f)
    metadata_reformatted = {}
    for cohort_name, info in metadata.items():
        for participant_id, participant_metadata in info.items():
            metadata_reformatted[(cohort_name, participant_id)] = (
                participant_metadata
            )
            metadata_reformatted[(cohort_name, participant_id)]["cohort"] = (
                cohort_name
            )
    return metadata_reformatted


particpant_metadata = load_particpant_metadata(
    path / "participant_metadata.json"
)
pprint(particpant_metadata[("HA", "001")])

In [ ]:
recording_metadata = {"measurement_condition": "laboratory"}

In [ ]:
particpant_metadata

In [ ]:
from mobgap.data import LabExampleDataset

example_data = LabExampleDataset()
example_data

In [ ]:
single_trial = example_data[4]
single_trial

In [ ]:
single_trial.participant_metadata

In [ ]:
single_trial.data_ss.head()

In [ ]:
loaded_data = {}
participant_metadata_for_dataset_from_data = {}
recording_metadata_for_dataset_from_data = {}

for d in all_data_files:
    recording_identifier = d.name.split(".")[0].split("_")
    cohort, participant_id = d.parts[-3:-1]
    loaded_data[(cohort, participant_id, *recording_identifier)] = {
        "LowerBack": load_data(d)
    }
    participant_metadata_for_dataset_from_data[
        (cohort, participant_id, *recording_identifier)
    ] = particpant_metadata[(cohort, participant_id)]
    recording_metadata_for_dataset_from_data[
        (cohort, participant_id, *recording_identifier)
    ] = recording_metadata

In [ ]:
loaded_data

In [ ]:
from mobgap.data import GaitDatasetFromData

dataset_from_data = GaitDatasetFromData(
    loaded_data,
    sampling_rate_hz,
    participant_metadata_for_dataset_from_data,
    recording_metadata_for_dataset_from_data,
)
dataset_from_data

In [ ]:
single_trial = dataset_from_data.get_subset(
    level_0="HA", level_1="002", level_3="Test11"
)[0]

In [ ]:
dataset_from_data.get_subset(
    level_0="HA", level_1="001", level_3="Test5"
)

In [ ]:
single_trial.data_ss.head()


In [ ]:
from mobgap.pipeline import MobilisedPipelineHealthy

pipe = MobilisedPipelineHealthy().run(single_trial)
pipe.per_wb_parameters_.drop(columns="rule_obj").T

In [ ]:
from mobgap.data.base import BaseGaitDataset


class CsvExampleData(BaseGaitDataset):
    def __init__(
        self,
        base_path: Path,
        *,
        groupby_cols: Optional[Union[list[str], str]] = None,
        subset_index: Optional[pd.DataFrame] = None,
    ) -> None:
        self.base_path = base_path
        super().__init__(groupby_cols=groupby_cols, subset_index=subset_index)

    def _path_from_index(self) -> Path:
        self.assert_is_single(None, "_path_from_index")
        g = self.group_label
        return (
            self.base_path
            / g.cohort
            / g.participant_id
            / f"{g.time_measure}_{g.test}_{g.trial}.csv"
        )

    def create_index(self) -> pd.DataFrame:
        all_data_files = sorted(list(self.base_path.rglob("*.csv")))
        index = []
        for d in all_data_files:
            recording_identifier = d.name.split(".")[0].split("_")
            cohort, participant_id = d.parts[-3:-1]
            index.append((cohort, participant_id, *recording_identifier))
        return pd.DataFrame(
            index,
            columns=[
                "cohort",
                "participant_id",
                "time_measure",
                "test",
                "trial",
            ],
        )

In [ ]:
csv_data = CsvExampleData(path)
csv_data

In [ ]:
path

In [ ]:
from mobgap.data.base import ParticipantMetadata


class CsvExampleData(BaseGaitDataset):
    # Our constant values:
    sampling_rate_hz: float = 100
    measurement_condition = "laboratory"
    recording_metadata = {"measurement_condition": "laboratory"}

    def __init__(
        self,
        base_path: Path,
        *,
        groupby_cols: Optional[Union[list[str], str]] = None,
        subset_index: Optional[pd.DataFrame] = None,
    ) -> None:
        self.base_path = base_path
        super().__init__(groupby_cols=groupby_cols, subset_index=subset_index)

    def _path_from_index(self) -> Path:
        self.assert_is_single(None, "_path_from_index")
        g = self.group_label
        return (
            self.base_path
            / g.cohort
            / g.participant_id
            / f"{g.time_measure}_{g.test}_{g.trial}.csv"
        )

    def create_index(self) -> pd.DataFrame:
        all_data_files = sorted(list(self.base_path.rglob("*.csv")))
        index = []
        for d in all_data_files:
            recording_identifier = d.name.split(".")[0].split("_")
            cohort, participant_id = d.parts[-3:-1]
            index.append((cohort, participant_id, *recording_identifier))
        return pd.DataFrame(
            index,
            columns=[
                "cohort",
                "participant_id",
                "time_measure",
                "test",
                "trial",
            ],
        )

    @property
    def participant_metadata(self) -> ParticipantMetadata:
        self.assert_is_single(None, "participant_metadata")
        return particpant_metadata[
            (self.group_label.cohort, self.group_label.participant_id)
        ]

    # data loading:
    @property
    def data(self) -> dict[str, pd.DataFrame]:
        # Data loading is only allowed, once we have just a single recording selected.
        self.assert_is_single(None, "data")
        return {"LowerBack": load_data(self._path_from_index())}

    @property
    def data_ss(self) -> pd.DataFrame:
        self.assert_is_single(None, "data_ss")
        return self.data["LowerBack"]

In [ ]:
csv_data = CsvExampleData(path)
csv_data

In [ ]:
single_trial = csv_data.get_subset(
    cohort="HA", participant_id="001", test="Test5"
)[0]
single_trial

In [ ]:
pipe = MobilisedPipelineHealthy().run(single_trial)
pipe.per_wb_parameters_.drop(columns="rule_obj").T

In [ ]:
import numpy as np
import pandas as pd
from mobgap.data import LabExampleDataset

lab_example_data = LabExampleDataset(reference_system="INDIP")
long_trial = lab_example_data.get_subset(
    cohort="MS", participant_id="001", test="Test11", trial="Trial1"
)
long_trial_gs = long_trial.reference_parameters_.wb_list

long_trial_gs

In [ ]:
from mobgap.pipeline import iter_gs

for gs, data in iter_gs(long_trial.data_ss, long_trial_gs):
    # Note that the key to access the id is called "wb_id" here, as we loaded the WB from the reference system.
    # If this is an "actual" gait sequences, as calculated by one of the GSD algorithms, the key would be "gs_id".
    print("Gait Sequence: ", gs)
    print("Expected N-samples in gs: ", gs.end - gs.start)
    print("N-samples in gs: ", len(data))
    print("First sample of gs:\n", data.iloc[0], end="\n\n")